# Evaluasi menggunakan format BioASQ dari hasil RAG

Sampai saat ini, mungkin temen2 udah coba buat code RAG yang bisa dipakai sampai model RAGnya bisa **men-generate jawaban** dari suatu pertanyaan. Nah untuk evaluasi ini, ternyata aku butuh code RAG temen2 hanya sampai **menghasilkan dokumen-dokumen yang terkait** saja. Karena kita ingin menyamakan tujuan eksperimen di papernya yaitu menggunakan **model BioBERT** untuk menjawab pertanyaan tsb.

##### 0. Preparation

In [2]:
import pandas as pd
import tqdm
import os
import json

from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter
from openai import OpenAI
from pinecone import Pinecone

character_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n", ". ", " ", ""], chunk_size=250, chunk_overlap=50)
token_splitter = SentenceTransformersTokenTextSplitter(tokens_per_chunk=250, chunk_overlap=50)

client = OpenAI()

def embedding_function(text, model="text-embedding-ada-002"):

    response = client.embeddings.create(
        input=text,
        model=model
    )

    embedding = response.data[0].embedding
    return embedding
 
pc = Pinecone(api_key=os.environ['PINECONE_API_KEY'])
index = pc.Index("rag-kak")


##### Step 1: Looping pertanyaan untuk dicari dokumen-dokumen terkaitnya

Di repo ini sudah aku siapkan data pertanyaan di *data/test_data/BioASQ-test-yesno-rag-11b.json*. Codenya sudah aku siapkan, dan aku minta temen2 untuk menghubungkan code ini dengan **metode RAG masing2**.

In [ ]:
df = pd.read_json('data/test_data/BioASQ-test-yesno-rag-11b.json')

output_json = {
    'data': [
        {
            'paragraphs': [],
            'title': 'BioASQ11b'
        }
    ],
    'version': 'BioASQ11b'
}

for i in tqdm(df.index):
    query = df.loc[i, 'questions']
    id = df.loc[i, 'ids']

    # TODO: Function ini yang dihubungkan ke code RAG temen2
    # Expected dari retrieved_documents isinya list of vectors dari index 'rag-kak' di Pinecone, yang paling relevan dengan query.
    # Ex: ["aaa", "bbb", ] (ambil top 10 docs)
    retrieved_documents = get_documents(query)
    context = '\n'.join(retrieved_documents)

    dct = {
        'qas': [
            {
                'id': f"{id}_001",
                'question': query
            }
        ],
        'context': context
    }
    output_json['data'][0]['paragraphs'].append(dct)

with open('data/test_data/BioASQ-test-yesno-11b.json', 'w') as outfile: 
    json.dump(output_json, outfile, indent=2)


Hasil dari code tersebut adalah list of pertanyaan with contextnya yang disimpan di *data/test_data/BioASQ-test-yesno-11b.json*.

##### Step 2: Evaluasi menggunakan standard dataset BioASQ dan model BioBERT

Disini kita akan pakai code dari pihak BioASQ untuk melakukan evaluasi sekaligus mengeluarkan metriksnya. Step2 detailnya sebagai berikut: 

Note: (disaranin banget jalanin di colab/kaggle karena butuh GPU dan Java)

#### 2a. Pertama temen2 bisa clone, dan masuk ke direktorinya:

```bash
git clone https://github.com/dmis-lab/biobert-pytorch
cd biobert-pytorch/question-answering/
```

#### 2b. Ubah sedikit bagian code di run_yesno.py:
Pada line 579 saat load Tokenizer, tambahkan
```bash
use_fast=False
```
pada parameternya.

#### 2c. Pindahkan data train dan test (question dan answer) ke direktori ini
Copy dua data dari repo sebelumnya (rag-chromadb) ke dalam repo folder ini (biobert-pytorch/question-answering). Data yang dipindahkan adalah 

- *data/train_data/BioASQ-train-yesno-11b.json* 
- *data/test_data/BioASQ-test-yesno-11b.json* 
- *data/test_data/11B_golden.json*.

#### 2d. Jalankan command berikut untuk melakukan training dan testing:

```bash
!python run_yesno.py \
    --model_type bert \
    --model_name_or_path "dmis-lab/biobert-base-cased-v1.1-squad" \
    --do_train \
    --train_file "BioASQ-train-yesno-11b.json" \
    --per_gpu_train_batch_size 12 \
    --learning_rate 8e-6 \
    --num_train_epochs 3 \
    --max_seq_length 384 \
    --seed 0 \
    --output_dir "output/"
```

```bash
python run_yesno.py \
    --model_type bert \
    --model_name_or_path "dmis-lab/biobert-base-cased-v1.1-squad" \
    --do_eval \
    --predict_file "BioASQ-test-yesno-11b.json" \
    --golden_file "11B_golden.json" \
    --output_dir "output/"
```

#### 2e. Keluar hasilnya:

Jika berjalan lancar, contoh keluaran hasilnya akan seperti ini:

![Embedding Adaptors](assets/eval.jpg)

##### Selesai. Bisa dicatat nilai-nilainya